In [1]:
import pandas as pd
import numpy as np


tourney_seeds = pd.read_csv("./Data/TourneySeeds.csv")
rank = []
teams = []
for seed, team in tourney_seeds.loc[:,['Seed','Team']].values:
    rank.append(int(seed[1:3]))
tourney_seeds["NumericalSeeds"] = rank

In [2]:
tourney_compact_results = pd.read_csv("./Data/TourneyCompactResults.csv")

In [17]:
tourney_seeds.head()

,Season,Seed,Team,NumericalSeeds
0,1985,W01,1207,1
1,1985,W02,1210,2
2,1985,W03,1228,3
3,1985,W04,1260,4
4,1985,W05,1374,5


In [3]:
tourney_compact_results.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [3]:
result = pd.DataFrame({
        "Season": pd.concat([tourney_compact_results['Season'], tourney_compact_results['Season']], ignore_index=True),
        "Player": pd.concat([tourney_compact_results['Wteam'], tourney_compact_results['Lteam']], ignore_index=True),
        "Opponent": pd.concat([tourney_compact_results['Lteam'], tourney_compact_results['Wteam']], ignore_index=True),
        "Result": np.hstack([np.ones(tourney_compact_results.shape[0], dtype=int), np.zeros(tourney_compact_results.shape[0], dtype=int)])
    }, columns=["Result", "Season", "Player", "Opponent"])

In [4]:
t = pd.merge(result, tourney_seeds, left_on=['Season', 'Player'], right_on=['Season', 'Team'])
t = t[['Result', 'Season','Player', 'Opponent', 'NumericalSeeds']]
t.columns=['Result', 'Season','Player', 'Opponent', 'PlayerSeed']
result = t

In [5]:
t = pd.merge(result, tourney_seeds, left_on=['Season', 'Opponent'], right_on=['Season', 'Team'])
t = t[['Result', 'Season', 'Player', 'Opponent', 'PlayerSeed', 'NumericalSeeds']]
t.columns = ['Result', 'Season','Player', 'Opponent', 'PlayerSeed', 'OpponentSeed']
result = t

In [6]:
regular = pd.read_csv("./Data/RegularSeasonDetailedResults.csv")
regular.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [7]:
regular_rev = regular.ix[:, [0,1,4,5,2,3,7,21,22,23,24,25,26,27,28,29,30,31,32,33,8,9,10,11,12,13,14,15,16,17,18,19,20]]
new_regular = pd.DataFrame(np.vstack([regular_rev.values, 
                                   regular.ix[:, [0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]].values], 
                                 ), columns = ["Season","Daynum","Player","PlayerScore","Opponent","OpponentScore","Numot","Wfgm","Wfga","Wfgm3","Wfga3","Wftm","Wfta","Wor","Wdr","Wast","Wto","Wstl","Wblk","Wpf","Lfgm","Lfga","Lfgm3","Lfga3","Lftm","Lfta","Lor","Ldr","Last","Lto","Lstl","Lblk","Lpf"])

In [8]:
loc = []
revloc = []
for itm in regular['Wloc'].values:
    if itm=='N':
        loc.append(0)
        revloc.append(0)
    elif itm=='H':
        loc.append(1)
        revloc.append(2)
    else:
        loc.append(2)
        revloc.append(1)
new_regular['PlayerLoc']=np.hstack([revloc, loc])
new_regular['PlayerResult']=np.hstack([np.zeros(regular.shape[0], dtype=np.int), np.ones(regular.shape[0], dtype=np.int)])

In [9]:
new_regular.head()

,Season,Daynum,Player,PlayerScore,Opponent,OpponentScore,Numot,Wfgm,Wfga,Wfgm3,...,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf,PlayerLoc,PlayerResult
0,2003,10,1328,62,1104,68,0,22,53,2,...,18,14,24,13,23,7,1,22,0,0
1,2003,10,1393,63,1272,70,0,24,67,6,...,19,15,28,16,13,4,4,18,0,0
2,2003,11,1437,61,1266,73,0,22,73,3,...,29,17,26,15,10,5,2,25,0,0
3,2003,11,1457,50,1296,56,0,18,49,6,...,31,6,19,11,12,14,2,18,0,0
4,2003,11,1208,71,1400,77,0,24,62,6,...,13,17,22,12,14,4,4,20,0,0


Effective field goal percentage

In [9]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wfgm', 'Wfgm3', 'Wfga', 'Lfgm', 'Lfgm3', 'Lfga']].copy()
t["EfgO"] = np.require(t['Wfgm'].values, dtype=np.float)+np.require(t['Wfgm3'].values, dtype=np.float)/2
t["EfgO"] = t['EfgO']/t['Wfga']
t["EfgD"] = np.require(t['Lfgm'].values, dtype=np.float)+np.require(t['Lfgm3'].values, dtype=np.float)/2
t["EfgD"] = t['EfgD']/t['Lfga']
t = t.groupby(by=['Season', 'Player']).mean()
rst = pd.DataFrame({'EfgO': t['EfgO'], 'EfgD': t['EfgD']}, columns=['EfgO', 'EfgD'])
rst.head()

EfgO      EfgD
Season Player                    
2003   1102    0.584407  0.514142
       1103    0.536564  0.549098
       1104    0.475785  0.480099
       1105    0.457983  0.509308
       1106    0.481697  0.452774

In [10]:
feature = pd.DataFrame(tourney_seeds[['Season', 'Team']].values, columns=['Season', 'Player'])
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

Turnover percentage

In [11]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wfga', 'Wfta', 'Wto', 'Lfga', 'Lfta', 'Lto']].copy()
t["TopO"] = np.require(t['Wfga'].values+t['Wto'].values, dtype=np.float) + \
0.44* np.require(t['Wfta'].values, dtype=np.float)
t["TopO"] = t['Wto']/t['TopO']
t["TopD"] = np.require(t['Lfga'].values+t['Lto'].values, dtype=np.float) + \
0.44* np.require(t['Lfta'].values, dtype=np.float)
t["TopD"] = t['Lto']/t['TopD']
t = t.groupby(by=['Season', 'Player']).mean()
rst = pd.DataFrame({'TopO': t['TopO'], 'TopD': t['TopD']}, columns=['TopO', 'TopD'])
rst.head()

TopO      TopD
Season Player                    
2003   1102    0.193121  0.203236
       1103    0.159805  0.186897
       1104    0.167275  0.179498
       1105    0.207792  0.212924
       1106    0.213721  0.194026

In [12]:
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

Rebounding

In [13]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wor', 'Wdr', 'Lor', 'Ldr']].copy()
t["ORB"] = np.require(t["Wor"].values+t["Ldr"].values, dtype=np.float)
t["ORB"] = t['Wor']/t['ORB']
t["DRB"] = np.require(t["Wdr"].values+t["Lor"].values, dtype=np.float)
t["DRB"] = t['Wdr']/t['DRB']
t = t.groupby(by=['Season', 'Player']).mean()
rst = pd.DataFrame({'ORB': t['ORB'], 'DRB': t['DRB']}, columns=['ORB', 'DRB'])
rst.head()

ORB       DRB
Season Player                    
2003   1102    0.168235  0.630384
       1103    0.305803  0.626998
       1104    0.371256  0.686897
       1105    0.335166  0.641434
       1106    0.349480  0.679342

In [14]:
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

free throws

In [15]:
t = new_regular[['Season', 'Player', 'Opponent', 'Wfga', 'Wftm', 'Lfga', 'Lftm']].copy()
t['OFT'] = np.require(t['Wftm'].values, dtype=np.float)/np.require(t['Wfga'].values, dtype=np.float)
t['DFT'] = np.require(t['Lftm'].values, dtype=np.float)/np.require(t['Lfga'].values, dtype=np.float)
t = t.groupby(by=['Season', 'Player']).mean()
rst = pd.DataFrame({'OFT': t['OFT'], 'DFT': t['DFT']}, 
                   columns=['OFT', 'DFT'])
rst.head()

OFT       DFT
Season Player                    
2003   1102    0.291436  0.336057
       1103    0.342976  0.282092
       1104    0.263855  0.223414
       1105    0.255629  0.289471
       1106    0.199740  0.297459

In [16]:
feature = pd.merge(feature, rst, left_on=['Season', 'Player'], right_index=True)

In [18]:
feature.head()

,Season,Player,EfgO,EfgD,TopO,TopD,ORB,DRB,OFT,DFT
1154,2003,1328,0.512124,0.448150,0.155078,0.181439,0.347284,0.709854,0.234856,0.236844
1155,2003,1448,0.511972,0.463124,0.178941,0.167749,0.429724,0.687237,0.354094,0.221934
1156,2003,1393,0.515151,0.445717,0.160408,0.165338,0.385242,0.630790,0.274586,0.199656
1157,2003,1257,0.528861,0.450485,0.156505,0.205507,0.356053,0.664037,0.292504,0.329973
1158,2003,1280,0.521359,0.463462,0.197467,0.208120,0.383446,0.695785,0.220035,0.199428


## Feature normalization

In [17]:
from scipy import stats

cdf = dict()
hist, bin_edges = np.histogram(feature['EfgO'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['EfgO'] = stats.rv_discrete(name='EfgO', values=(bin_edges[:-1], hist))
feature['EfgO'] = cdf['EfgO'].cdf(feature['EfgO'].values)

In [18]:
hist, bin_edges = np.histogram(feature['EfgD'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['EfgD'] = stats.rv_discrete(name='EfgD', values=(bin_edges[:-1], hist))
feature['EfgD'] = cdf['EfgD'].cdf(feature['EfgD'].values)

hist, bin_edges = np.histogram(feature['TopO'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['TopO'] = stats.rv_discrete(name='TopO', values=(bin_edges[:-1], hist))
feature['TopO'] = cdf['TopO'].cdf(feature['TopO'].values)

hist, bin_edges = np.histogram(feature['TopD'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['TopD'] = stats.rv_discrete(name='TopD', values=(bin_edges[:-1], hist))
feature['TopD'] = cdf['TopD'].cdf(feature['TopD'].values)

hist, bin_edges = np.histogram(feature['ORB'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['ORB'] = stats.rv_discrete(name='ORB', values=(bin_edges[:-1], hist))
feature['ORB'] = cdf['ORB'].cdf(feature['ORB'].values)

hist, bin_edges = np.histogram(feature['DRB'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['DRB'] = stats.rv_discrete(name='DRB', values=(bin_edges[:-1], hist))
feature['DRB'] = cdf['DRB'].cdf(feature['DRB'].values)

hist, bin_edges = np.histogram(feature['OFT'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['OFT'] = stats.rv_discrete(name='DRB', values=(bin_edges[:-1], hist))
feature['OFT'] = cdf['OFT'].cdf(feature['OFT'].values)

hist, bin_edges = np.histogram(feature['DFT'].values, bins=10000, density=True)
hist/=hist.sum()
cdf['DFT'] = stats.rv_discrete(name='DFT', values=(bin_edges[:-1], hist))
feature['DFT'] = cdf['DFT'].cdf(feature['DFT'].values)

In [19]:
feature.head()

,Season,Player,EfgO,EfgD,TopO,TopD,ORB,DRB,OFT,DFT
1154,2003,1328,0.396552,0.164871,0.307112,0.545259,0.548491,0.747845,0.106681,0.506466
1155,2003,1448,0.394397,0.362069,0.784483,0.299569,0.994612,0.456897,0.964440,0.351293
1156,2003,1393,0.447198,0.134698,0.410560,0.270474,0.875000,0.024784,0.462284,0.169181
1157,2003,1257,0.664871,0.188578,0.340517,0.851293,0.635776,0.173491,0.653017,0.971983
1158,2003,1280,0.540948,0.370690,0.947198,0.884698,0.864224,0.567888,0.044181,0.165948


In [20]:
result['SeedDelta'] = (result['OpponentSeed']-result['PlayerSeed']+16.0)/31
result['SeedState'] = (17.0-result['PlayerSeed'].values)/16
result = pd.merge(result, feature, on=['Season', 'Player'])
result = pd.merge(result, feature, left_on=['Season', 'Opponent'], right_on=['Season', 'Player'], suffixes=["Ply", "Opp"])

In [8]:
# complete record test
result['SeedDelta'] = (result['OpponentSeed']-result['PlayerSeed']+16.0)/31

In [21]:
result.drop(['PlayerOpp', 'SeedState'], 1, inplace=True)
result = result.rename(columns={"PlayerPly": "Player"})

Parameter tunning

In [9]:
result.tail()

,Result,Season,Player,Opponent,PlayerSeed,OpponentSeed,SeedDelta
4095,0,2016,1396,1234,10,7,0.419355
4096,1,2016,1458,1338,7,10,0.612903
4097,1,2016,1458,1462,7,2,0.354839
4098,0,2016,1451,1462,15,2,0.096774
4099,1,2016,1462,1451,2,15,0.935484


In [11]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.regularizers import l2, l1, l1l2
from keras.optimizers import SGD

model = Sequential([
    Dense(1, input_dim=1, W_regularizer=l1l2(0.002, 0.005), init='lecun_uniform'),
    Activation('sigmoid')
])

opt = SGD(lr=0.125, momentum=0.92, decay=1e-4)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_crossentropy'])

Using Theano backend.
WARNING (theano.sandbox.cuda): GPU device gpu0 will be initialized, and used if a GPU is needed. However, no computation, nor shared variables, will be implicitly moved to that device. If you want that behavior, use the 'device' flag instead.
Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN Version is too old. Update to v5, was 3007.)


In [12]:
model.fit(X, np.expand_dims(Y, 1), nb_epoch=70, batch_size=2048, validation_data=(Xt, np.expand_dims(Yt, 1)), verbose=1)

Train on 3564 samples, validate on 134 samples
Epoch 1/70
3564/3564 [==============================] - 0s - loss: 0.7002 - binary_crossentropy: 0.7000 - val_loss: 0.6973 - val_binary_crossentropy: 0.6973
Epoch 2/70
3564/3564 [==============================] - 0s - loss: 0.6979 - binary_crossentropy: 0.6977 - val_loss: 0.6943 - val_binary_crossentropy: 0.6943
Epoch 3/70
3564/3564 [==============================] - 0s - loss: 0.6936 - binary_crossentropy: 0.6936 - val_loss: 0.6908 - val_binary_crossentropy: 0.6908
Epoch 4/70
3564/3564 [==============================] - 0s - loss: 0.6890 - binary_crossentropy: 0.6889 - val_loss: 0.6873 - val_binary_crossentropy: 0.6873
Epoch 5/70
3564/3564 [==============================] - 0s - loss: 0.6849 - binary_crossentropy: 0.6844 - val_loss: 0.6839 - val_binary_crossentropy: 0.6839
Epoch 6/70
3564/3564 [==============================] - 0s - loss: 0.6806 - binary_crossentropy: 0.6797 - val_loss: 0.6804 - val_binary_crossentropy: 0.6804
Epoch 7/70


In [33]:
np.expand_dims(Y, 1)

array([[1],
       [0],
       [0],
       ..., 
       [1],
       [1],
       [1]])

In [48]:
Yp=model.predict_proba(Xt)

 32/134 [======>.......................] - ETA: 0s

In [66]:
X

array([[ 0.51612903,  0.14224138,  0.97952586, ...,  0.4612069 ,
         0.92780172,  0.22952586],
       [ 0.03225806,  0.14224138,  0.97952586, ...,  0.27155172,
         0.61314655,  0.54525862],
       [ 0.38709677,  0.67025862,  0.04956897, ...,  0.27155172,
         0.61314655,  0.54525862],
       ..., 
       [ 0.87096774,  0.54741379,  0.13469828, ...,  0.76293103,
         0.92780172,  0.81357759],
       [ 0.74193548,  0.82974138,  0.5       , ...,  0.95043103,
         0.92780172,  0.14331897],
       [ 0.87096774,  0.44073276,  0.04956897, ...,  0.4612069 ,
         0.71012931,  0.54525862]])

In [10]:
seg = result[(result['Season'])<2013].copy()
X = seg.ix[:, 6:7].values
Y = seg.ix[:, 0].values
test = result[(result['Season']==2013)].copy()
Xt = test.ix[:, 6:7].values
Yt = test.ix[:, 0].values

In [48]:
cver.best_params_

{'min_samples_leaf': 4, 'n_estimators': 110}

In [49]:
rf = RandomForestClassifier(n_estimators=110, min_samples_leaf=4)
rf.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=110, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Test on test set

In [31]:
Yp = model.predict_proba(Xt)

 32/134 [======>.......................] - ETA: 0s

In [52]:
#keeps the best auc
from sklearn.metrics import log_loss

log_loss(Yt, Yp)

0.61295923585826095

In [32]:
from sklearn.metrics import log_loss
log_loss(Yt, Yp, eps=1e-7)

0.60233398560267781

In [30]:
#keeps the best auc
from sklearn.metrics import roc_auc_score

roc_auc_score(Yt, Yp[:,1])

0.73535308531967036

In [34]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Yt, Yp)

0.72978391623969707

# Submit result

In [38]:
submission = pd.read_csv("./Data/sample_submission.csv")
content = []
for row in submission.iterrows():
    content.append(map(int, row[1].ix[0].split('_')))

In [39]:
submission = pd.DataFrame(content, columns=['Season', 'Player', 'Opponent'])

In [40]:
# take features from existing data
t = pd.merge(submission, tourney_seeds, left_on=['Season', 'Player'], right_on=['Season', 'Team'])
t = t.ix[:, [0,1,2,5]]
t.columns = ['Season', 'Player', 'Opponent', 'PlayerSeed']
t = pd.merge(t, tourney_seeds, left_on=['Season', 'Opponent'], right_on=['Season', 'Team'])
t = t.ix[:, [0, 1, 2, 3, 6]]
t.columns = ['Season', 'Player', 'Opponent', 'PlayerSeed', 'OpponentSeed']
t['SeedDelta'] = (t['OpponentSeed']-t['PlayerSeed']+16.0)/31
t['SeedState'] = (17.0-t['PlayerSeed'].values)/16
t = pd.merge(t, feature, on=['Season', 'Player'])
t = pd.merge(t, feature, left_on=['Season', 'Opponent'], right_on=['Season', 'Player'], suffixes=["Ply", "Opp"])
t.drop('PlayerOpp', 1, inplace=True)
t = t.rename(columns={"PlayerPly": "Player"})

In [47]:
X = t.ix[:, 5:23].values
Y = rf.predict_proba(X)

In [48]:
t['Prediction']=Y[:, 1]

In [49]:
id = []
pred = []
for row in t.iterrows():
    id.append("{0}_{1}_{2}".format(int(row[1].ix[0]), int(row[1].ix[1]), int(row[1].ix[2])))
    pred.append(row[1].ix[23])

In [50]:
submit = pd.DataFrame({"id":id, "pred":pred}, columns=['id', 'pred'])

In [51]:
submit.head()

,id,pred
0,2013_1103_1107,0.527143
1,2013_1103_1112,0.275000
2,2013_1107_1112,0.075000
3,2013_1103_1125,0.395238
4,2013_1107_1125,0.309286


In [52]:
submit.to_csv("result.csv", index=False)

Try to calculate loss locally

In [53]:
submission = t[['Season', 'Player', 'Opponent', 'Prediction']]

In [54]:
submission.shape

(9112, 4)

In [55]:
t1 = pd.merge(submission, tourney_compact_results, left_on=['Season', 'Player', 'Opponent'], right_on=['Season', 'Wteam', 'Lteam'])
t2 = pd.merge(submission, tourney_compact_results, left_on=['Season', 'Player', 'Opponent'], right_on=['Season', 'Lteam', 'Wteam'])

In [56]:
t1['Label']=1
t2['Label']=0
submission = pd.concat([t1, t2], ignore_index=True)
log_loss(submission['Label'].values, submission['Prediction'].values)

0.2889324328931448